# 実験3-4 自己符号化器による特徴表現の獲得

ここでは自己符号化器（オートエンコーダ）に基づく教師なし学習を考えてみます．
オートエンコーダは，入力と出力のペアを学習するモデルではありますが，入出力のペアには全く同じデータを与えるところに特徴があります．
ある入力を与えたときに，入力と同じ出力を出すネットワークに意味があるのかという話になりますが，その中間表現には意味が出てきます．
例えば中間層の次元を絞ったモデルでうまく学習できたときには，絞った次元でデータを表現し，その表現を使うともとのパターンを復元するだけの情報を持っているという
ことになりますから，データに依存した次元圧縮が可能になっていると捉えることができます．

ここでは，

1. オートエンコーダによる MNIST の潜在表現の分析
2. 変分オートエンコーダ(VAE) によるデータの生成と分析

について考えたいと思います．

## 実験準備

この実験では

 - `MNIST`, `Fachin-MNIST`, `CIFAR10` データセット
 
を用います．これらは結構なディスク容量を消費するので，IED で実験を行う場合，少々小細工が必要となります．

実験2-4 と同様に，画像のデータセットをダウンロードする先を設定します．
IED では `/usr/local/class/media/dataset` にありますので，
```code:python
datadir = `/usr/local/class/media/dataset/`
```
を指定してください．自分の家や google Colab で頑張る場合は，適宜設定してください．

In [ ]:
import os

# GPU の指定
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # 使うGPUを指定

# データディレクトリの指定 (IED の場合は，下のほうをコメントアウト)
datadir = './data' # 自前のディレクトリに置く場合
# datadir = '/usr/local/class/media/dataset' # IED で実験する場合

## オートエンコーダの構成

最初にオートエンコーダを考えたいと思います．このネットワークは符号化器(エンコーダ: encoder) と復号器(デコーダ: decoder) を組み合わせたモデルと考えることができます．
このエンコーダとデコーダの接合部分が潜在表現となります．
もでるとしてつくると下記のようなモデルとなります．

In [1]:
# シンプルなオートエンコーダの構築
import torch
import torch.nn as nn

# オートエンコーダモデル
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 32),
            nn.ReLU(),
            nn.Linear(32, 128),
            nn.ReLU(),
            nn.Linear(128, 784),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

ここでは MNIST を取り扱うために エンコーダとデコーダ，それぞれのネットワーク構成を

- エンコーダ: 784-128-32-2
- デコーダ: 2-32-128-784

と対称になるようなモデル構成で考え，それぞれの層に非線形関数である `ReLU()` と `Sigmoid()` を設定しています．
デコーダの最終層の非線形関数をシグモイド関数に設定しているのは，もとの MNIST のデータセットが [0, 1] の区間で定義されたデータであることに由来しています．

### データセットの準備

このモデルに `MNIST` データセットを学習させることを考えます．
まずはデータセットを準備します．

In [ ]:
# データセット準備
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

# 1. データセットの前処理設定
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # 28x28 を 784次元に平坦化
])

# MNIST データセットの読み込み
mnist_train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
dataloader = DataLoader(mnist_train_dataset, batch_size=128, shuffle=True)

### オートエンコーダの学習

次に学習させます．学習は，クラス判別とはことなり，エンコーダへ入力した信号と，デコーダの吐き出した出力信号との比較になります．
このとき，これらが同じになるようにするのがオートエンコーダのキモの部分なので，自分との違いを損失関数に設定する必要があります．
この場合は `nn.MSELoss()` 関数を使います．

また，学習に用いる最適化手法は `SGD` を使っても良いのですが，ここでは時間短縮のため `Adam` と呼ばれる方法を取ります．

In [ ]:
import torch.optim as optim
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# モデル定義
model = AutoEncoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 20
history = []
# モデル学習
for epoch in range(num_epochs):
    total_loss = 0
    for x, _ in dataloader:
        x = x.to(device)
        optimizer.zero_grad()
        _, decoded = model(x)
        loss = criterion(decoded, x)
        history.append(loss.item())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(dataloader.dataset):.4f}")



In [ ]:
plt.semilogy(history, label="loss")
plt.xlabel("iterations")
plt.ylabel("loss")
plt.grid()


無事に学習できているようなので中間層の表現を眺めていきます．
中間層の表現を得るには，モデルのエンコーダ部分を収集します．

In [6]:
encoded_data = []
labels = []

with torch.no_grad():
    for x, targets in dataloader:
        x = x.to(device)
        encoded, _ = model(x)  # エンコードされた値を取得
        encoded_data.append(encoded.cpu())
        labels.append(targets)

encoded_data = torch.cat(encoded_data)  # 全データを結合
labels = torch.cat(labels)

最後に集めたデータは，２次元のデータなので，これをラベルとともに散布図とします．

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
scatter = plt.scatter(encoded_data[:, 0], encoded_data[:, 1], c=labels, cmap='tab10', alpha=0.7)
plt.colorbar(scatter, ticks=range(10), label='Digit Label')
plt.xlabel('Encoded Dimension 1')
plt.ylabel('Encoded Dimension 2')
plt.title('2D Encoded Representation of MNIST')
plt.grid(True)


それでは，上記(0, 0)の付近であればなんかパターンがいくつか混ざって混沌とした状況なので，サンプルしてパターンを生成してみます．

In [ ]:
# 生成したいデータの数
num_generation = 10

# サンプルデータを生成 mu を中心に sigma の標準偏差な乱数生成
mu = torch.tensor([0, 0])
sigma = torch.tensor([1, 1])
std_rand = torch.randn(num_generation, 2)
sample_embeddeing = (sigma * std_rand + mu).to(device)


with torch.no_grad():
    generated = model.decoder(sample_embeddeing).view(-1, 28, 28).cpu()
    
plt.figure(figsize=(20, 4))
for i in range(num_generation):
    ax = plt.subplot(2, num_generation, i+1)
    plt.imshow(generated[i], cmap='gray')
    ax.axis('off')

## 変分自己符号化器(VAE)

変分自己符号化器（Variational Auto Encoder: VAE）は，AEの亜種で，潜在表現を既知の確率分布で表すことができるようにしたモデルです．
潜在表現を既知の確率分布（通常は多次元正規分布）に表現させようとすることにしているため，逆に潜在空間の上で，その分布に基づいた乱数をふることで
乱数からデータを生成させることができるようになります．
これに基づいて画像などのデータ生成をさせるというのがVAEの主たる使い方になります．（Diffusion モデルのご先祖だと思ってもらっても良いです）

モデル構成は下記の通りになります．
ここでは潜在空間は２次元の空間となるので２次元の正規分布が間にはさまるような形になっていると思ってもらって大丈夫です．
符号化器を正規分布化するために，中心パラメータを制御する `fc_mu` と分散を制御する `fc_logvar` が付け加えられています．

`reparameterize()` 関数は，ネットワークの中心で正規分布化するための関数で，ここのなかで乱数を生成させ，デコードさせることで新奇なパターンをどんどん生成させることができるようにします．
（この生成方法はリパラメトリゼーショントリックと呼ばれます）
したがって，学習時にはこの正規分布がどこにあるのかを獲得する必要が出てきます．
このため `forward()` 関数では，復号化したデータ以外にも，分布の中心 `mu` と `logvar` を返すような形になります．

In [9]:
import torch
import torch.nn as nn

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        # Encoder: 潜在空間の平均と標準偏差を出力
        self.encoder = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(32, 2)      # 2次元分の平均 μ
        self.fc_logvar = nn.Linear(32, 2)  # 2次元分の対数分散 log(σ^2)

        # Decoder: 潜在空間からデータを再構成
        self.decoder = nn.Sequential(
            nn.Linear(2, 32),
            nn.ReLU(),
            nn.Linear(32, 128),
            nn.ReLU(),
            nn.Linear(128, 784),
            nn.Sigmoid()
        )

    def reparameterize(self, mu, logvar):
        # 再パラメータ化トリック
        std = torch.exp(0.5 * logvar)  # 標準偏差
        epsilon = torch.randn_like(std)  # 標準正規分布からサンプリング
        return mu + std * epsilon # 学習された正規分布へマッピング

    def forward(self, x):
        # エンコーダ
        encoded = self.encoder(x)
        mu = self.fc_mu(encoded)
        logvar = self.fc_logvar(encoded)
        
        # 再パラメータ化
        z = self.reparameterize(mu, logvar)
        
        # デコーダ
        reconstructed = self.decoder(z)
        return reconstructed, mu, logvar

学習では，復号データと入力データの差分だけではなく，符号化データが与えた確率分布に従っているように細工が必要となります．このため，単純な再構成誤差だけでなく，KLダイバージェンスと呼ばれる項を損失関数に追加します．

In [ ]:
import torch.optim as optim

def vae_loss(reconstructed, x, mu, logvar):
    # 再構成損失 (MSE または BCE)
    reconstruction_loss = nn.functional.binary_cross_entropy(reconstructed, x, reduction='sum')
    
    # KLダイバージェンス
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return reconstruction_loss + kl_divergence


# デバイス設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# モデルと最適化器の準備
vae = VAE().to(device)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)

# 学習ループ
num_epochs = 20
vae.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch, (x, _) in enumerate(dataloader):
        x = x.to(device)
        
        # 順伝播
        reconstructed, mu, logvar = vae(x)
        
        # 損失計算
        loss = vae_loss(reconstructed, x, mu, logvar)
        
        # 逆伝播とパラメータ更新
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(dataloader.dataset):.4f}")



In [ ]:
vae.eval()
encoded_data = []
labels = []

with torch.no_grad():
    for x, y in dataloader:
        x = x.to(device)
        _, mu, _ = vae(x)  # 平均値を潜在変数として使用
        encoded_data.append(mu.cpu())
        labels.append(y)

encoded_data = torch.cat(encoded_data)
labels = torch.cat(labels)

# 散布図のプロット
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 8))
scatter = plt.scatter(encoded_data[:, 0], encoded_data[:, 1], c=labels, cmap='tab10', alpha=0.7)
plt.colorbar(scatter, ticks=range(10), label='Digit Label')
plt.xlabel('Latent Dimension 1')
plt.ylabel('Latent Dimension 2')
plt.title('Latent Space Representation (VAE)')
plt.grid(True)
plt.show()

それでは，２次元の潜在空間上で適当な２点を選んで線分を構成し，その線上のデータがどのようなデータに再構成されるかを眺めてみましょう．

In [ ]:
# 2つの潜在ベクトル間を補間
z1 = (1.5 * torch.randn(1, 2)).to(device)  # 潜在ベクトル1
z2 = (1.5 * torch.randn(1, 2)).to(device)  # 潜在ベクトル2
t = torch.linspace(0, 1, steps=10).unsqueeze(1).to(device)
interpolated_z = t * z1 + (1 - t) * z2

# データ生成
vae.eval()
with torch.no_grad():
    interpolated_images = vae.decoder(interpolated_z)

# 可視化
plt.figure(figsize=(10, 2))
for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.imshow(interpolated_images[i].view(28, 28).cpu().numpy(), cmap='gray')
    plt.axis('off')
plt.suptitle('Latent Space Interpolation')
plt.show()

## 実験 3-4

1. AE, VAE から得られた潜在空間表現に k-meas 法を施し，どのようなクラスタが得られるかを図示しなさい．
2. AE, VAE の構成に畳み込み演算を入れた場合，潜在空間がどのようになるかを図示しなさい．
3. `Fashion-MNIST` と `CIFAR10` の潜在空間表現がどのようになるかを図示しなさい．